In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization
import matplotlib.pyplot as plt
import random
from sklearn.metrics import accuracy_score, log_loss
from scipy import optimize
from keras import losses
from keras import backend as K
import tensorflow as tf

%matplotlib notebook

# Preprocessing

In [11]:
data = pd.read_csv('leaf.csv', header=None)
data.drop([1],inplace=True,axis=1)
X = data.values[:,1:]
Y_first = data.values[:,0]
Y = []
out = np.zeros(shape=[36])
for i in range(np.shape(Y_first)[0]):
    out[int(Y_first[i])-1]=1
    Y.append(out)
    out = np.zeros(shape=[36])
Y = np.array(Y)
# min_max_scaler = preprocessing.MinMaxScaler()
# X_scaled = min_max_scaler.fit_transform(X)
# df_normalized = pd.DataFrame(X_scaled)

r,c = np.shape(X)

total_samples = list(np.arange(0,r,1))
test_samples = random.sample(total_samples,int(np.round(len(total_samples)*0.2)))
train_samples = list(set(total_samples) - set(test_samples))

# Splitting Data

In [55]:
trainX = X[train_samples]
trainX_scaled = (trainX-np.min(trainX))/(np.max(trainX)-np.min(trainX))
trainY = Y[train_samples]
trainY_comp = Y_first[train_samples]
print(trainY)

testX = X[test_samples]
testX_scaled = (testX-np.min(testX))/(np.max(testX)-np.min(testX))
testY = Y[test_samples]
testY_comp = Y_first[test_samples]
print(len(trainX[0]))
print(len(trainY[0]))

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
14
36


In [113]:
def logloss(true_label, predicted, eps=1e-15):
  p = np.clip(predicted, eps, 1 - eps)
  if true_label == 1:
    return -np.log(p)
  else:
    return -np.log(1 - p)

#  Neural Network

In [120]:
def nn_objective(weights):
    num_inputs = 14
    num_outputs = 36

    hidden_layer_neurons = 1
    
    weights_all = num_inputs*hidden_layer_neurons + hidden_layer_neurons*num_outputs
    bias_all = hidden_layer_neurons+num_outputs
    iw_size = num_inputs*hidden_layer_neurons
    nn_weights = weights[:weights_all]
    bias_all = weights[weights_all:]
    iw = weights[0:iw_size]
    ow = weights[iw_size:weights_all]
    b1 = bias_all[:]
    w1 = np.reshape(iw,(num_inputs, hidden_layer_neurons))
    w2 = np.reshape(ow,(hidden_layer_neurons, num_outputs))

    l1 = np.maximum(np.dot(trainX, w1), 0)
    x = np.dot(l1, w2)
    l2 = np.array(np.exp(x)/ sum(np.exp(x)))
    #predict_out = tf.convert_to_tensor(np.array(l2).astype('float32'))
    #actual_output = tf.convert_to_tensor((np.array(trainY[0:2]).astype('float32')))
    #er = losses.categorical_crossentropy(actual_output,predict_out)
    er = []
    for i in range(np.shape(trainY)[0]):
        er_i = 0
        for j in range(np.shape(trainY)[1]):
            er_i += logloss(np.array(trainY[i,j]),np.array(l2[i,j]))
        er.append(er_i)
    er = np.mean(er)
    print(weights[0:5])
    return er



In [124]:
num_inputs = 14
num_outputs = 36

hidden_layer_neurons = 1

tw = (num_inputs*hidden_layer_neurons)+(hidden_layer_neurons*num_outputs)
w0 = np.random.random((1,tw))
s = np.ones((1,tw))*10
res = optimize.basinhopping(nn_objective, w0[0], niter=10, T=1, stepsize=s[0],disp=True)
                            

[ 0.31414774  0.53165008  0.34854392  0.2674222   0.10005571]
[ 0.31414776  0.53165008  0.34854392  0.2674222   0.10005571]
[ 0.31414774  0.53165009  0.34854392  0.2674222   0.10005571]
[ 0.31414774  0.53165008  0.34854394  0.2674222   0.10005571]
[ 0.31414774  0.53165008  0.34854392  0.26742222  0.10005571]
[ 0.31414774  0.53165008  0.34854392  0.2674222   0.10005572]
[ 0.31414774  0.53165008  0.34854392  0.2674222   0.10005571]
[ 0.31414774  0.53165008  0.34854392  0.2674222   0.10005571]
[ 0.31414774  0.53165008  0.34854392  0.2674222   0.10005571]
[ 0.31414774  0.53165008  0.34854392  0.2674222   0.10005571]
[ 0.31414774  0.53165008  0.34854392  0.2674222   0.10005571]
[ 0.31414774  0.53165008  0.34854392  0.2674222   0.10005571]
[ 0.31414774  0.53165008  0.34854392  0.2674222   0.10005571]
[ 0.31414774  0.53165008  0.34854392  0.2674222   0.10005571]
[ 0.31414774  0.53165008  0.34854392  0.2674222   0.10005571]
[ 0.31414774  0.53165008  0.34854392  0.2674222   0.10005571]
[ 0.3141

[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.30557462 -0.45771119  0.30617758  0.25909621  0.09084125]
[ 0.3055

[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.38192977 -0.41215063  0.27156483  0.19898835  0.04194069]
[ 0.3819

[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.82409845 -0.10650811  0.38567664  0.39276367  0.29007331]
[ 0.80856188 -0.14535335  0.38677965  0.43440782  0.33462136]
[ 0.8085

[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.93475761 -0.27701916  0.41023496  0.7999471   0.75496771]
[ 0.9347

...

[  5.63188571   7.71894883   2.23935909  10.85074891   3.885566  ]
[  5.6318857    7.71894884   2.23935909  10.85074891   3.885566  ]
[  5.6318857    7.71894883   2.2393591   10.85074891   3.885566  ]
[  5.6318857    7.71894883   2.23935909  10.85074893   3.885566  ]
[  5.6318857    7.71894883   2.23935909  10.85074891   3.88556602]
[  5.6318857    7.71894883   2.23935909  10.85074891   3.885566  ]
[  5.6318857    7.71894883   2.23935909  10.85074891   3.885566  ]
[  5.6318857    7.71894883   2.23935909  10.85074891   3.885566  ]
[  5.6318857    7.71894883   2.23935909  10.85074891   3.885566  ]
[  5.6318857    7.71894883   2.23935909  10.85074891   3.885566  ]
[  5.6318857    7.71894883   2.23935909  10.85074891   3.885566  ]
[  5.6318857    7.71894883   2.23935909  10.85074891   3.885566  ]
[  5.6318857    7.71894883   2.23935909  10.85074891   3.885566  ]
[  5.6318857    7.71894883   2.23935909  10.85074891   3.885566  ]
[  5.6318857    7.71894883   2.23935909  10.85074891   3.88556

[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153201]
[ 17.34543604   2.84800768   2.98992798   9.377656    -0.96153

[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.7646007 ]
[ 16.00046282   7.99449772  11.49998273  10.06868662  -7.76460

[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945658]
[ 35.45723501  13.93881319  17.40913853  12.46625217 -10.07945

[ 34.69256024  32.11672435  17.62961553   8.67301647  -4.17722981]
[ 34.69256024  32.11672435  17.62961553   8.67301647  -4.17722981]
[ 34.69256024  32.11672435  17.62961553   8.67301647  -4.17722981]
basinhopping step 9: f nan trial_f nan accepted 1  lowest_f 5.00195
[ 31.36307045  31.63356762  22.80363003  14.58095754   4.82519234]
[ 31.36307046  31.63356762  22.80363003  14.58095754   4.82519234]
[ 31.36307045  31.63356763  22.80363003  14.58095754   4.82519234]
[ 31.36307045  31.63356762  22.80363005  14.58095754   4.82519234]
[ 31.36307045  31.63356762  22.80363003  14.58095755   4.82519234]
[ 31.36307045  31.63356762  22.80363003  14.58095754   4.82519236]
[ 31.36307045  31.63356762  22.80363003  14.58095754   4.82519234]
[ 31.36307045  31.63356762  22.80363003  14.58095754   4.82519234]
[ 31.36307045  31.63356762  22.80363003  14.58095754   4.82519234]
[ 31.36307045  31.63356762  22.80363003  14.58095754   4.82519234]
[ 31.36307045  31.63356762  22.80363003  14.58095754   4.8251

In [81]:
weights = res.x

def nn_objective_test(weights):
    num_inputs = 14
    num_outputs = 36

    hidden_layer_neurons = 15
    
    np.random.seed(4)
    iw_size = num_inputs*hidden_layer_neurons
    iw = weights[0:iw_size]
    ow = weights[iw_size:]
    w1 = np.reshape(iw,(num_inputs, hidden_layer_neurons))
    w2 = np.reshape(ow,(hidden_layer_neurons, num_outputs))
#     w1 = 2*np.random.random((num_inputs, hidden_layer_neurons)) - 1
#     w2 = 2*np.random.random((hidden_layer_neurons, num_outputs)) - 1

    weights = np.append(np.array(w1.flatten()),np.array(w2.flatten()))
    learning_rate = 0.2 # slowly update the network
    for epoch in range(1):
        l1 = np.maximum(np.dot(testX, w1), 0)
        #l2 = 1/(1 + np.exp(-(np.dot(l1, w2))))
        x = np.dot(l1, w2)
        l2 = np.array(np.exp(x)/ sum(np.exp(x)))
        predict_out = np.argmax(l2,axis=1)
        er = log_loss(np.array(testY),np.array(l2)).mean()
    return er

test_miss = nn_objective_test(weights)
print(test_miss)

22.9149941501
